# CNN classifier

### In this file we are going to test the CNN classifier on the dataset.

The Goal is to answer the question that is the CNN able to detect the onBody devices from offBody.

And if so how? We will visualize the output of each layer to analyze how the optimizer extract features and potentially provide explanations of what those features are!

In [5]:
## you can run pip3 install -r requirements.txt to install all the packages
## but you need to install tensorflow and keras manually

import pymongo
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import threading, os, sys
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # try to use CPU only

# addin path to import IQ module
sys.path.append('../')
import src.IQ as IQ


In [4]:
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/admin")
BLE = myclient["BLE"]

def query(collection, filter:dict, addFrameColumn=True):
    df =  pd.DataFrame(list(collection.find(filter)))
    if addFrameColumn:
        df['frame'] = df.apply(lambda x: x['I'] + np.dot(x['Q'],1j), axis=1)
    return df.copy()

In [6]:
# Create a StandardScaler object
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
scaler2 = MinMaxScaler(feature_range=(0, 1))
# Assuming 'new_csv_features' is your data

def to_complex(amplitudes, angles):
    return np.array([r * np.exp(1j * theta) for r, theta in zip(amplitudes, angles)])

def normalized(row):
    row = np.array(row)
    return scaler2.fit_transform(scaler.fit_transform(row.reshape(-1, 1)))

def fft_normalized(row, threshold = 0.5):
    tempfft = np.fft.fft(row)

    amp = np.abs(tempfft)
    angle = np.angle(tempfft)
    #filtering criteria
    filtering = amp > np.average(amp)*threshold
    
    amp[filtering] = 0
    angle[filtering] = 0
    temp = to_complex(amplitudes = amp,angles =  angle)
    ###### option 1 to go back to the time domain
    # temp = np.real(np.fft.ifft(temp))
    # # # uncomment to see the effect of frequency filtering
    # # plt.plot(row) 
    # # plt.plot(temp)
    # # plt.show()
    # # plt.close()
    # return normalized(temp)

    ###### option 2 to concat real and imaginary
    ##### try to teach the ifft at the same time
    return np.concatenate([normalized(np.real(temp)), normalized(np.imag(temp))])


In [6]:
from sklearn.model_selection import train_test_split

filtering = {''}
df = query(BLE['onBody'], {'pos':'static','antenna_side':'left'})
min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:2000])
print(len(df['frame'][0]))


iq = IQ.IQ(Fc=2439810000+.1e4)

def configCreator(downSampleRate = 1, cutoff = 1e6):
    downSampleRate= max(downSampleRate, 1)
    return {                                      
            iq.gradient:{},
            iq.unwrapPhase:{},
            iq.phase:{}, 
            iq.butter:{'Fs': iq.Fs/downSampleRate, "cutoff": cutoff},
            iq.downSample:{'downSampleRate':downSampleRate, "shift": 0},
            iq.demodulate:{'Fs': iq.Fs},
           } 

methods = configCreator(downSampleRate=  1)
df['data'] = iq.apply(methods = methods, frame = df)

############### Defing and normalizing the input #############
# df['normalized_input_feature'] = df['data'].apply(lambda x: fft_normalized(x, threshold = 1))
df['normalized_input_feature'] = df['data'].apply(lambda x: normalized(x))
##################################################################################

X_train, X_test, y_train, y_test = train_test_split(df['normalized_input_feature'], df['dvc'], test_size=0.2, random_state=42)

2000


In [7]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

X_train = tf.convert_to_tensor(X_train.tolist())
X_test =  tf.convert_to_tensor(X_test.tolist())
y_train =  tf.convert_to_tensor(y_train.tolist())
y_test = tf.convert_to_tensor(y_test.tolist())

data_shape = len(df['data'][0])

y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

2023-12-03 19:24:14.283876: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:24:14.295224: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:24:14.295364: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, Dropout

model = Sequential()
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(y_test_encoded.shape[1], activation='softmax'))  # 13 classes


# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_encoded, epochs=64, batch_size=64, validation_data=(X_test, y_test_encoded))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test_encoded)
print(f'Test accuracy: {accuracy}')

Epoch 1/64


2023-12-03 19:24:20.641816: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-12-03 19:24:21.807362: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-03 19:24:21.980693: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-03 19:24:22.470711: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f2678016c50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-03 19:24:22.470725: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-12-03 19:24:22.480182: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-03 19:24:22.548893: I ./tensorflow/compiler/jit/device_comp

37/37 [==============================] - 6s 25ms/step - loss: 2.1713 - accuracy: 0.2394 - val_loss: 1.5538 - val_accuracy: 0.3929
Epoch 2/64
37/37 [==============================] - 0s 12ms/step - loss: 1.4636 - accuracy: 0.3803 - val_loss: 1.3656 - val_accuracy: 0.4337
Epoch 3/64
37/37 [==============================] - 0s 12ms/step - loss: 1.3688 - accuracy: 0.4012 - val_loss: 1.2986 - val_accuracy: 0.4541
Epoch 4/64
37/37 [==============================] - 0s 12ms/step - loss: 1.2999 - accuracy: 0.4485 - val_loss: 1.2722 - val_accuracy: 0.4915
Epoch 5/64
37/37 [==============================] - 0s 12ms/step - loss: 1.2197 - accuracy: 0.4974 - val_loss: 1.1317 - val_accuracy: 0.5901
Epoch 6/64
37/37 [==============================] - 0s 12ms/step - loss: 1.0134 - accuracy: 0.6256 - val_loss: 0.9249 - val_accuracy: 0.6956
Epoch 7/64
37/37 [==============================] - 0s 13ms/step - loss: 0.8299 - accuracy: 0.6827 - val_loss: 0.7929 - val_accuracy: 0.7041
Epoch 8/64
37/37 [======

In [9]:
z = model(X_test[0:100])
zz = pd.DataFrame(np.array(z))

# # num_rows = zz.shape[0]

# # # Plot each row
# # for i in range(num_rows):
# #     # plt.figure(figsize=(8, 4))  # Adjust the size as needed
# #   plt.plot(zz.iloc[i, :])

# # plt.show()
zz


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,5.663389e-13,1.036578e-06,9.465654e-01,1.365405e-09,5.113464e-04,5.899991e-12,7.239871e-05,1.472596e-11,1.298047e-06,5.264936e-02,1.467320e-11,8.651663e-05,1.127383e-04
1,2.165502e-13,3.077097e-08,9.897810e-01,2.663336e-11,1.285725e-03,1.334636e-13,5.305730e-06,1.017601e-12,3.980785e-07,7.011780e-03,3.535215e-13,4.966882e-04,1.419029e-03
2,1.434041e-13,3.969575e-11,2.642625e-04,3.109609e-12,1.579288e-09,4.310125e-11,6.958469e-03,1.965252e-10,2.102963e-05,4.794110e-10,9.927562e-01,9.545925e-10,9.835367e-11
3,8.550330e-15,7.682666e-17,1.272818e-05,2.214879e-13,5.162982e-02,2.489643e-17,5.700727e-05,6.679854e-11,2.882977e-02,7.946622e-11,2.773445e-12,2.827469e-01,6.367238e-01
4,2.849594e-14,1.693014e-18,1.700155e-08,2.087776e-14,8.311757e-01,2.214770e-11,4.614596e-07,1.300196e-12,1.629056e-01,2.063968e-14,3.225678e-13,3.522926e-08,5.918115e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,4.121018e-14,7.426518e-10,1.474479e-10,8.329284e-06,4.778592e-06,9.990011e-01,2.930930e-05,1.280917e-11,9.058390e-04,1.605302e-07,5.027884e-05,8.267602e-12,2.037589e-07
96,2.351291e-17,1.876025e-21,6.037377e-09,1.749948e-15,1.408963e-04,1.033843e-24,3.049115e-08,1.540783e-13,1.761054e-04,1.043934e-15,5.297049e-21,3.732800e-01,6.264029e-01
97,1.456763e-12,4.381385e-17,6.391666e-04,3.127062e-13,3.838752e-04,1.445084e-12,1.337453e-03,1.083235e-12,9.939400e-01,2.186604e-17,8.419930e-09,5.918226e-05,3.640262e-03
98,3.180556e-15,8.312894e-09,9.997762e-01,7.603330e-12,5.905248e-05,9.413374e-15,6.180565e-09,7.497221e-13,9.913057e-08,1.501619e-04,7.231874e-14,3.080129e-06,1.138869e-05


In [10]:
df = query(BLE['offBody'], {'pos':'1','antenna':'1', 'dvc':'7'})
# min_length = df['frame'].apply(len).min()
df['frame'] = df['frame'].apply(lambda x: x[:min_length])
df['data'] = iq.apply(methods = methods, frame = df)

X_test = tf.convert_to_tensor(df['data'].tolist())

z = model(X_test)
zz = pd.DataFrame(np.array(z))

num_rows = zz.shape[0]

# Plot each row
for i in range(num_rows):
    # plt.figure(figsize=(8, 4))  # Adjust the size as needed
  plt.plot(zz.iloc[i, :])

plt.show()

ValueError: Input 0 of layer "sequential" is incompatible with the layer: expected shape=(None, 2000, 1), found shape=(467, 4022)